# Dynamic Programming Algorithm to solve the shortest path problem with parking and travel times

Import of necessary packages

In [12]:
from collections import defaultdict
import time

Define the graph G. The graph has nodes and arcs. To each node a time interval is assigned which specifies the time during which parking is allowed. Each arcs has a distance, which is in this case the duration to travel between the corresponding nodes. Moreover, the predecessors and successors of each node are saved in a separate class attribute.

In [13]:
class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = list()
        self.edges_out = defaultdict(list)
        self.edges_in = defaultdict(list)
        self.distance = dict()
        self.f_dep_t = dict()
        self.f_park_t = dict()

    def add_node(self, value, feasible_parking_time):
        self.nodes.add(value)
        self.f_park_t[value] = feasible_parking_time

    def add_edge(self, from_node, to_node, distance, feasible_dep_time):
        self.edges.append((from_node, to_node))
        self.edges_out[from_node].append(to_node)
        self.edges_in[to_node].append(from_node)
        self.distance[(from_node, to_node)] = distance
        self.f_dep_t[(from_node, to_node)] = feasible_dep_time

This function build the graph. The data for the working example have to be set in this function.

In [15]:
def build_graph():
    """
    Define the network for the example in this function. Note that an additional "artifical"
    node has to be specified at the very end of the network, where no restrictions in terms of
    parking exist.
    :return: graph containing the data of the specified example
    """
    # Nodes of the example
    g = Graph()
    
    # define the feasible parking times at each node
    parking_times = {
        1: [0, float('inf')],
        2: [18, 30],
        3: [35, 45],
        4: [64, 75],
        5: [35, 50],
        6: [64, 75],
        7: [0, float('inf')],
        8: [0, float('inf')] # artificial node
    }
    
    # define the feasible travel times between all nodes
    travel_times = {
        (1, 2): [10, 15],
        (1, 3): [10, 25],
        (1, 5): [15, 25],
        (2, 4): [28, 35],
        (2, 6): [26, 34], 
        (2, 7): [32, 60],
        (3, 2): [35, 50],
        (3, 4): [43, 50],
        (4, 7): [70, 99],
        (5, 2): [30, 80],
        (5, 6): [48, 55],
        (6, 7): [72, 80],
        (7, 8): [0, 0]
    }
    
    # define the duration to travel between nodes
    duration = {
        (1, 2): 10,
        (1, 3): 20,
        (1, 5): 25,
        (2, 4): 35,
        (2, 6): 35, 
        (2, 7): 30,
        (3, 2): 15,
        (3, 4): 20,
        (4, 7): 30,
        (5, 2): 20,
        (5, 6): 25,
        (6, 7): 10,
        (7, 8): 0 # artificial node
    }
    
    arcs = list(duration.keys())
    
    # initialize the nodes
    for i in range(1, 9):
        g.add_node(i, parking_times[i])

    # Arcs of the example
    for arc in arcs:
        g.add_edge(*arc, duration[arc], travel_times[arc]) # (from node, to node, distance/duration, feasible travel times)

    return g

The function **get_route_for_arc** returns for a given solution the route to a specific node. 

In [16]:
def get_route_for_arc(arc: tuple, optimal_i: dict):
    """
    This function returns all arcs used in order to arrive at a given arc for a specified solution set
    :param arc: The arc for which preceeding arcs shall be determined
    :param optimal_i: The current solution found by the algorithm, i.e. for all f(i,j) the optimal
    preceeding node.
    :return: Set of arcs preceeding the arc specified in the functions arguments
    """
    check_route_for = arc
    route = list()
    i = check_route_for[0]
    j = check_route_for[1]
    try:
        while True:
            curr_arc = (i, j)
            i = optimal_i[curr_arc]
            j = curr_arc[0]
            route.append((i, j))
            if i == 1:
                break
        return [i for i in reversed(route)]
    # Direct links from sink nodes have no optimal predecessor, hence they don´t appear in the solution
    # set. Therefore, when keys are not found in the dictionary, and empty list will be returned
    except KeyError:
        return []

The function **calc_min_earliest_arrival_time** determines for a given graph $g$ the earliest feasible arrival time $f(j, k)$ at a node $j$. This is done by first iterating over all nodes that have a direct connection to node $j$. In each iteration, the dynamic programming technique is used, by retrieving the earliest feasible arrival time at a given node $i$ which has been determined earlier. This algorithm hence has to run sequencially, as for a given node $j$ the earliest feasible arrival times of all its predecessors have to be calculated alread. 

The $f(i, j)$ is then used to determine the parking time at $i$. This is done using the function **check_for_feasibility** which will be explained in the furth course of this script.

With the parking time $p_i$ at $i$, the earliest feasible arrival time at this node as well as the distance between $i$ and $j$ the arrival time at $j$ when $i$ preceeds $j$ can be determined: $f'(j,k) = f(i,j) + p_i + t_{ij}$, where f'(j,k) refers to a potential solution to the problem $f(j,k) = min[f(i,j) + p_i + t_{ij}$\]

In the end, the minimum of the potential solutions is selected and the results for $f(j,k)$, $p_i$ and $opt i$ are saved.

In [17]:
def calc_min_earliest_arrival_time(g, j, k, earliest_feasible_arrival_times,
                                   opt_is, parking):
    # earliest_feasible_arrival_times = earliest_feasible_arrival_times.copy()
    # opt_is = opt_is.copy()
    # parking = parking.copy()
    if j == 1:
        # we have initialized all arcs from the sink already
        return earliest_feasible_arrival_times, opt_is, parking

    pot_earliest_feasible_arrival_times = []
    arcs_to_j = [ij[0] for ij in g.edges if ij[1] == j]
    print("The following nodes move into {}: {}".format(j, arcs_to_j))
    inter_parking = []
    for i in arcs_to_j:
        f_ij = earliest_feasible_arrival_times[i, j]
        print("f({}, {}) is: {}".format(i, j, f_ij))
        p_i = check_for_feasibility(g, i, j, f_ij, opt_is, earliest_feasible_arrival_times, parking)
        f_ij = earliest_feasible_arrival_times[i, j]
        print("p_{}:".format(i), p_i)
        t_ij = g.distance[i, j]
        # parking[i, j] = p_i
        inter_parking.append(p_i)
        print("current earliest_feasible_arrival_times: ", earliest_feasible_arrival_times)
        print("current parking times: ", parking)
        if p_i is not False and type(p_i) == int:
            print("f_{}; p_{}; t_{}: ".format((i, j), i, (i, j)), f_ij, "; ", p_i, "; ", t_ij)
            pot_earliest_feasible_arrival_times.append(f_ij + p_i + t_ij)
        else:
            pot_earliest_feasible_arrival_times.append(float('inf'))
        print("pot_earliest_feasible_arrival_times", pot_earliest_feasible_arrival_times)
    earliest_feasible_arrival_times[j, k] = min(pot_earliest_feasible_arrival_times)
    parking[j, k] = inter_parking[pot_earliest_feasible_arrival_times.index(min(pot_earliest_feasible_arrival_times))]
    print("best i", pot_earliest_feasible_arrival_times.index(min(pot_earliest_feasible_arrival_times)))
    opt_is[j, k] = arcs_to_j[
        pot_earliest_feasible_arrival_times.index(min(pot_earliest_feasible_arrival_times))]

    return earliest_feasible_arrival_times, opt_is, parking

In [18]:
def do_recursion(g, i, j, earliest_feasible_at, opt_is, earliest_feasible_arrival_times, parking):
    """
    Recursion for identifying a parking time p_i when in succeeding links f(j,k) a situation occurs,
    where condition (equation) 3 from the paper is met, i.e. a node is visited before the parking
    is allowed as well as before the transition to node k is possible. In this case, parking time
    in preceeding nodes has to be increased. This works in a backtracking manner, i.e. if parking
    cannot increased in preceeding node i because parking is not allowed there yet but transition
    to the next node j is, then the preceeding node for i is checked. If at no preceeding node
    parking is possible, then the solution set will be omitted (set to infinity)
    :param i:
    :param j:
    :param earliest_feasible_at:
    :param opt_is:
    :param earliest_feasible_arrival_times:
    :return:
    """
    # specify time window for parking
    print("so lets do the recursion")
    alpha_i = g.f_park_t[i][0]
    beta_i = g.f_park_t[i][1]

    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i, j)][0]
    delta_ij = g.f_dep_t[(i, j)][1]

    previous_arcs = get_route_for_arc((i, j), opt_is)
    print(previous_arcs)
    if len(previous_arcs) == 0:
        return float('inf')
    else:
        # this time is missing on node j in order to be able to park or directly move to k
        time_gap = min([alpha_i - earliest_feasible_at, gamma_ij - earliest_feasible_at])
        if time_gap < 0:
            time_gap = max([alpha_i - earliest_feasible_at, gamma_ij - earliest_feasible_at])
        print("time_gap", time_gap)

        curr_prev_arc = previous_arcs[-1]
        print(curr_prev_arc)
        f_i1_i2 = earliest_feasible_arrival_times[curr_prev_arc]
        new_p_i = check_for_feasibility(g, *curr_prev_arc, f_i1_i2, opt_is, earliest_feasible_arrival_times,
                                        parking, add_park_time=time_gap)
        if not new_p_i == float('inf'):
            new_earliest_feasible_at = f_i1_i2 + new_p_i + g.distance[curr_prev_arc]
            print("new f({}): ".format((i, j)), new_earliest_feasible_at, "new_p{}: ".format(i), new_p_i)
            earliest_feasible_arrival_times[i, j] = new_earliest_feasible_at
            # parking[curr_prev_arc] = new_p_i
            parking[i, j] = new_p_i
        else:
            new_earliest_feasible_at = earliest_feasible_at
        return check_for_feasibility(g, i, j, new_earliest_feasible_at, opt_is,
                                     earliest_feasible_arrival_times, parking)

In [19]:
def check_for_feasibility(g, i, j, earliest_feasible_at, opt_is, earliest_feasible_arrival_times,
                          parking, add_park_time=None):
    print("Check feasibility for usage of arc ({}) for earliest feasible time {}".format((i, j), earliest_feasible_at))
    if earliest_feasible_at == float('inf'):
        print("Infeasible solution found. Recalculate the best path for node {} proceeding to {}".format(i,j))
        earliest_feasible_arrival_times, opt_is, parking = \
            calc_min_earliest_arrival_time(g, i, j, earliest_feasible_arrival_times, opt_is, parking)
        latest_arc = get_route_for_arc((i, j), opt_is)
        if len(latest_arc) == 0:
            return False
        else:
            return parking[i, j]

    # specify time window for parking
    alpha_i = g.f_park_t[i][0]
    beta_i = g.f_park_t[i][1]

    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i, j)][0]
    delta_ij = g.f_dep_t[(i, j)][1]

    # check first equation:
    if alpha_i <= delta_ij:
        # pass
        max_p_i = min(beta_i - earliest_feasible_at, delta_ij - earliest_feasible_at)
        p_i = []
        for feas_p_i in range(max_p_i + 1):
            if (alpha_i <= earliest_feasible_at <= beta_i) and (
                    gamma_ij <= earliest_feasible_at + feas_p_i <= delta_ij):
                p_i.append(feas_p_i)
        print("feasible parking times at {} when moving to {}: {}".format(i, j, p_i))
        if len(p_i) > 0:
            print("1st cond satisfied")
            if add_park_time:
                print("additional parking time at node {} is needed: ".format(i), add_park_time)
                new_p_i = p_i[0] + add_park_time
                print("This sums up to: ", new_p_i)
                if new_p_i < max_p_i:
                    print('which is still a feasible solution. Proceed with p_{} = {}'.format(i, new_p_i))
                    return new_p_i
                else:
                    print("which is not a feasible solution. ")
                    # So when we move here we set f(2,7) to infinity, but what we really want to do is set
                    # f(1,2) to infinit and with this new insight calculate a new f(2,7): How can we proceed here?
                    earliest_feasible_arrival_times[i,j] = float('inf')
                    return float('inf')
            else:
                print("Proceed with least possible parking time :", p_i[0])
                return p_i[0]

    # check second equation
    if (beta_i - earliest_feasible_at <= 0) and (gamma_ij <= earliest_feasible_at <= delta_ij):
        print("2nd cond satisfied")
        p_i = 0
        return p_i
    # check third equation
    elif (earliest_feasible_at < gamma_ij <= alpha_i) or (
            earliest_feasible_at < alpha_i < gamma_ij < beta_i) or (
            earliest_feasible_at < beta_i < gamma_ij):
        a = do_recursion(g, i, j, earliest_feasible_at, opt_is, earliest_feasible_arrival_times, parking)
        print("recursion is done, result is: ", a)
        return a

    # check fourth equation
    elif gamma_ij < earliest_feasible_at < alpha_i <= delta_ij:
        print("4th cond satisfied")
        p_i = 0
        return [p_i]

    # check fifth equation
    elif delta_ij < earliest_feasible_at:
        print("5th cond satisfied")
        return [float('inf')]

    else:
        print(Warning("Some cases are not considered in feasibility check"))

In [20]:
def main():
    g = build_graph()
    earliest_feasible_arrival_times = dict()
    sink = min(g.nodes)
    terminal = max(g.nodes)
    optimal_predecessors = dict()
    optimal_parking = defaultdict(list)
    arcs_from_sink = [jk[1] for jk in g.edges if jk[0] == sink]
    print("initialize the arcs from the sink")
    for k in arcs_from_sink:
        earliest_feasible_arrival_times[sink, k] = 0
    for k in g.nodes.difference({sink}):
        print("The iteration over k is currently at: ", k)
        arcs_to_k = [jk[0] for jk in g.edges if jk[1] == k]
        print("The following nodes move into {}: {}".format(k, arcs_to_k))
        for j in arcs_to_k:
            earliest_feasible_arrival_times, optimal_predecessors, optimal_parking = \
                calc_min_earliest_arrival_time(g, j, k, earliest_feasible_arrival_times, optimal_predecessors, optimal_parking)

    return earliest_feasible_arrival_times, optimal_predecessors, optimal_parking


In [21]:
if __name__ == "__main__":
    start = time.time()
    earliest_feasible_arrival_times, opt_is, parking = main()
    print("Finished. Runtime was: {} \n".format(time.time() - start))
    print("""The result looks as follows: \n 
          The earliest feasible arrival times are: {} \n
          The optimal predecessor nodes per node are: {} \n 
          And the optimal parking durations at node i for an
          arc (j,k) are as follows: {}""".format(earliest_feasible_arrival_times,
                                                 opt_is, parking))

initialize the arcs from the sink
The iteration over k is currently at:  2
The following nodes move into 2: [1, 3, 5]
The following nodes move into 3: [1]
f(1, 3) is: 0
Check feasibility for usage of arc ((1, 3)) for earliest feasible time 0
feasible parking times at 1 when moving to 3: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1st cond satisfied
Proceed with least possible parking time : 10
p_1: 10
current earliest_feasible_arrival_times:  {(1, 2): 0, (1, 3): 0, (1, 5): 0}
current parking times:  defaultdict(<class 'list'>, {})
f_(1, 3); p_1; t_(1, 3):  0 ;  10 ;  20
pot_earliest_feasible_arrival_times [30]
best i 0
The following nodes move into 5: [1]
f(1, 5) is: 0
Check feasibility for usage of arc ((1, 5)) for earliest feasible time 0
feasible parking times at 1 when moving to 5: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1st cond satisfied
Proceed with least possible parking time : 15
p_1: 15
current earliest_feasible_arrival_times:  {(1, 2): 0, (1, 3): 0,